In [8]:
import cv2
import mediapipe as mp
import os
import json
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


Get Anotations

In [2]:
PATH_HaGRID = "public_img"

# there must be below path:
# - A folder called ann_subsample with json for each type of class
# - A folder called subsample, with a folder with the name of each class, 
#   each one with images representing the class

In [3]:
def get_annotations(file):
    json_annotations = json.load(open(file))
    json_annotations = [
        dict(ann, **{"name": f"{name}.jpg"})
        for name, ann in zip(json_annotations, json_annotations.values())
    ]
    annotations = pd.DataFrame(json_annotations)

    labels = list(annotations["labels"])
    bboxes = list(annotations["bboxes"])
    gestures = []

    for x in range(len(labels)):
        for l_b in zip(labels[x], bboxes[x]):
            if l_b[0] != "no_gesture":
                gestures.append(l_b)

    annotations["target"] = [g[0] for g in gestures]
    annotations["target_bbox"] = [g[1] for g in gestures]

    return annotations


def absFilePath(directory):
    for dirpath, _, filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))


annotations = pd.concat(
    [
        get_annotations(file)
        for file in absFilePath(os.path.join(PATH_HaGRID, "ann_subsample"))
    ],
    ignore_index=True,
)

annotations.set_index('name', inplace=True)
annotations['target'].value_counts()

mute               100
ok                 100
two_up             100
fist               100
palm               100
stop               100
four               100
two_up_inverted    100
peace              100
stop_inverted      100
call               100
like               100
one                100
rock               100
three              100
three2             100
peace_inverted     100
dislike            100
Name: target, dtype: int64

Let's go through each image and process their hands

In [104]:
from src.handDetector import HandDetector
import cv2


def get_croped(img, bbox, box_scale=1):
    h, w, c = img.shape

    x1 = bbox[0] * w
    x2 = x1 + bbox[2] * w
    y1 = bbox[1] * h
    y2 = y1 + bbox[3] * h

    x1, y1, x2, y2 = list(map(int, (x1, y1, x2, y2)))
    crop_image = img[y1:y2, x1:x2]

    return crop_image


detector = HandDetector()
results = []

for path, _, files in os.walk(os.path.join(PATH_HaGRID, "subsample")):
    for e, name in enumerate(files):
        # if e > 5:
        #     break
        if name.endswith(".jpg"):
            try:
                ann = annotations.loc[name]
                img = cv2.imread(os.path.join(path, name))

                img = get_croped(img, ann["target_bbox"])
                detector.fit(img)
                if detector.found_hands():
                    # There should be always 1 entry only (we are cropping)
                    # only one hand!
                    results.append(
                        [ann["target"], detector.get_positions_normalized()[0]]
                    )
                # img = detector.transform_connect_lines(img)

                # plt.imshow(img)
                # plt.show()
            except KeyError:
                print("Not found file: ", name)


Not found file:  000484ab-5fd0-49b8-9253-23a22b71d7b1-checkpoint.jpg


Save the file

In [105]:
target = [r[0] for r in results]
array = [r[1] for r in results]


df = pd.DataFrame(array)
df = df.round(4)
cols = df.columns
df['target'] = target

# Target should be at the beginning
cols = cols.insert(0, 'target')

df[cols].to_csv("out/lala.csv", index=False, header=False, mode='a')